# GSE162500 dataset analysis

## Loading packages, setting image parameters

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scirpy as ir
from matplotlib import pyplot as plt, cm as mpl_cm
from cycler import cycler


import seaborn as sns

from glob import glob
import tarfile
import anndata
import warnings

import scvelo as scv

# Additional functions
sns.set_style("ticks")

def beautiful_cmap(initial_cmap="Reds", grey_intensity=0.2, color_intencity=0.1):
    from matplotlib import cm
    from matplotlib.colors import ListedColormap
    import numpy as np
    
    cm_color = cm.get_cmap(initial_cmap, 128)
    cm_grey = cm.get_cmap("Greys", 128)
    
    c = ListedColormap(
        np.vstack(
            (cm_grey(np.linspace(grey_intensity, grey_intensity, 1)),
             cm_color(np.linspace(color_intencity, 1, 128)))
    ))
    
    return c

Reds = beautiful_cmap()

## Loading data

Download and unzip the datasets in advance as specified in the <mark> GSE162500_loading_data.md </mark>.

Create list of paths to load all samples

In [ ]:
samples0 = {
    "P34": {"source": ["Tumor"]},
    "P35": {"source": ["Tumor"]},
    "P42": {"source": ["Tumor"]},
    "P43": {"source": ["Tumor"]},
    "P46": {"source": ["Tumor"]},}
samples1 = {
    "P47": {"source": ["Tumor"]},
    "P55": {"source": ["Tumor"]},
    "P57": {"source": ["Tumor", "Blood"]},}
samples2 = {
    "P58": {"source": ["Tumor", "Blood"]},
    "P60": {"source": ["Tumor", "Juxta", "Blood"]},
    "P61": {"source": ["Tumor", "Juxta", "Blood"]},}   

In [ ]:
paths = []
for sample, sample_meta in samples0.items():
    for s in sample_meta['source']:
        gex_file = f"data/GSE162500_RAW/{sample}_{s}_raw_feature_bc_matrix"
        tcr_file = "No annotation"
        paths.append((sample, s, gex_file, tcr_file))
        
for sample, sample_meta in samples1.items():
    for s in sample_meta['source']:
        gex_file = f"data/GSE162500_RAW/{sample}_{s}_raw_feature_bc_matrix"
        tcr_file = f"data/GSE162500_RAW/{sample}_{s}_filtered_contig_annotations.csv"
        paths.append((sample, s, gex_file, tcr_file))
        
for sample, sample_meta in samples2.items():
    for s in sample_meta['source']:
        gex_file = f"data/GSE162500_RAW/{sample}_{s}_filtered_feature_bc_matrix"
        tcr_file = f"data/GSE162500_RAW/{sample}_{s}_filtered_contig_annotations.csv"
        paths.append((sample, s, gex_file, tcr_file))

Load all samples to adatas list. Read dataframes with TCR information if needed 

In [ ]:
adatas = []
for patient, s, gex_file, tcr_file in paths:
    print(patient, s)
    adata = sc.read_10x_mtx(gex_file)
    if tcr_file != "No annotation":
        adata_tcr = ir.io.read_10x_vdj(tcr_file)
        ir.pp.merge_with_ir(adata, adata_tcr)
    adata.obs["patient"] = patient
    adata.obs["source"] = s
    adata.obs["sample"] = f"{patient}_{s}"
    adata.obs.index = [barcode.split("-")[0] + f":{patient}_{s}" for barcode in adata.obs.index]
    adata.var_names_make_unique()
    adatas.append(adata)

Merge anndata objects using concat (**join = 'outer'** to take all columns)

In [ ]:
# Merge anndata objects
adata = anndata.concat(adatas, join = 'outer')

In [ ]:
adata.obs

## Preparing data for analysis

Filtering out genes found in less than 10 cells.

Filtering obviously bad cells (if the cell contains less than 100 counts, most likely this barcode corresponds to a drop into which the cell did not fall, but extracellular DNA was sequenced)

In [ ]:
print(adata.shape)
sc.pp.filter_genes(adata, min_cells=10)
print(adata.shape)
sc.pp.filter_cells(adata, min_genes=100)
print(adata.shape)

Write filtered data to <mark> GSE162500_adata_all_samples_filtered.h5ad </mark> for later use

In [ ]:
adata.write("data/GSE162500_adata_all_samples_filtered.h5ad")

### Filter cells with low counts

In [ ]:
ax = sns.histplot(np.log(adata.X.sum(axis=1).A.T[0]), bins=100)
plt.axvline(np.log(np.e**6.5))

It can be seen that there are 2 peaks on the histogram reflecting the distribution of couts by cells. The right peak represents cells suitable for analysis, and the left peak represents cells with too few counts.

Filter cells with less than 665 counts

In [ ]:
sc.pp.filter_cells(adata, min_genes=np.e**6.5)
print(adata.shape)

In [ ]:
ax = sns.histplot(np.log(adata.X.sum(axis=1).A.T[0]), bins=100)

### Filtering out cells with too many mitochondrial genes

Cells with too high a percentage of mitochondrial genes are highly likely to be damaged and unsuitable for analysis.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs.head()

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter = False, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 4000, :]
adata = adata[adata.obs.pct_counts_mt < 15, :]

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

### Find and remove doublets

When two cells fall into one drop on one ball with reagents, doublets are formed - 2 cells corresponding to 1 barcode. Due to the mixed signal, they can interfere with further analysis.

The scrublet algorithm will be used to search for doublets. Search is perforned per datch

In [ ]:
import scanpy.external as sce

In [ ]:
sce.pp.scrublet(adata, batch_key = "sample")

In [ ]:
adata = adata[adata.obs["doublet_score"] < 0.2, :].copy()

In [ ]:
adata_prepared = adata.copy()

### Normalization, log-transformation, selection of the most variable genes, PCA, clustering, UMAP

The following steps are required for clustering and visualization

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
adata_nocorr = adata.copy()
sc.pp.highly_variable_genes(
    adata_nocorr,
    n_top_genes=3000
)
adata_nocorr.raw = adata_nocorr
adata_nocorr = adata_nocorr[:, adata_nocorr.var.highly_variable]

sc.pp.scale(adata_nocorr)
sc.tl.pca(adata_nocorr, n_comps = 30)

sc.pp.neighbors(
    adata_nocorr,
    n_pcs=30,
    n_neighbors=20,
    knn=True
)

sc.tl.leiden(adata_nocorr)

sc.tl.umap(adata_nocorr)

In [ ]:
sc.pl.umap(adata_nocorr, color=["leiden", "patient", 'source', 'MKI67'],
           title=["Clusters", "Batch", 'Source'], ncols=2, wspace=0.4, frameon=False, cmap=Reds)

In [ ]:
def barplot(adata, cluster_key, batch_key):
    import matplotlib.pyplot as plt
    import seaborn
    
    fig_width = len(adata.obs[cluster_key].cat.categories) * 0.3
    fig, ax = plt.subplots(dpi=150, figsize=(fig_width, 2))
    
    colors = dict(zip(adata.obs[batch_key].cat.categories,
                      adata.uns[f"{batch_key}_colors"]))
    
    sizes = adata.obs.groupby([batch_key, cluster_key]).size()
    props = sizes.groupby(level=1).apply(lambda x: 100 * x / x.sum())
    props.index = props.index.droplevel(level=0)
    props = pd.DataFrame({'props': props,
                          f'{batch_key}': props.index.droplevel(level=1),
                          f'{cluster_key}': props.index.droplevel(level=0)
                         })
    props = props.pivot(columns=cluster_key, index=batch_key).T
    
    props.index = props.index.droplevel(0)
    props.fillna(0, inplace=True)
    props.plot.bar(stacked=True, width=1,
                   edgecolor="black", ax=ax, color=colors)
    plt.xticks(rotation=90)
    ax.set_xlabel("")
    ax.legend(loc=(1.01, 0.45), edgecolor="white")

In [ ]:
barplot(adata=adata_nocorr, cluster_key="leiden", batch_key="patient")

The batch effect is noticeable, as different samples are unevenly distributed across clusters

### Batch correction (harmony)

In [ ]:
import symphonypy as sp

In [ ]:
adata_harmony = adata.copy()

sc.pp.highly_variable_genes(
    adata_harmony,
    n_top_genes=3000,
    batch_key="sample"
)
adata_harmony.raw = adata_harmony
adata_harmony = adata_harmony[:, adata_harmony.var.highly_variable]

sc.pp.scale(adata_harmony, max_value=10)
sc.tl.pca(adata_harmony, n_comps = 30)

#sc.external.pp.harmony_integrate(adata_harmony, key="sample", max_iter_harmony=20)

sp.pp.harmony_integrate(adata_harmony, key="sample", max_iter_harmony=20)

# `theta` отвечает за силу интеграции
# `max_iter_harmony` отвечает за максимальное число итераций до тех пор, пока алгоритм не сойдётся

sc.pp.neighbors(
    adata_harmony,
    use_rep="X_pca_harmony",
    n_pcs=30
)

sc.tl.leiden(adata_harmony)

sc.tl.umap(adata_harmony)
     

In [ ]:
sc.pl.umap(adata_harmony, color=["leiden", "patient", 'source', 'MKI67'],
           title=["Clusters", "Batch", 'Source'], ncols=2, wspace=0.4, frameon=False, cmap=Reds)

In [ ]:
barplot(adata=adata_harmony, cluster_key="leiden", batch_key="patient")

After harmony, the clusters were mixed between patients, so you can work further with the dataset obtained with it.

## Filtration of T lymphocytes (CD3+)

In [ ]:
sc.set_figure_params(figsize = (10, 7.5))
sns.set_style("ticks")

In [ ]:
sc.pl.umap(adata_harmony, color=['leiden',"CD3E", "EPCAM", "CD14", "MS4A1", "MKI67"], size=10, ncols=2, cmap=Reds, legend_loc = 'on data', frameon=False)

EPCAM - epithelial cell marker

CD14 macrophage marker

MS4A1 - B-lymphocyte marker

MKI67 - cycling cell marker

Need to filter 9, 13, 15 and 17 clusters

In [ ]:
T_clusters = ["0", "1", "2", "3", "4", "5", "6", '7', "8", '10', '11', '12', '14', '16']
adata_T = adata_harmony[adata_harmony.obs.leiden.isin(T_clusters)]

adata_T = adata_T.raw.to_adata()
adata_T.uns["log1p"]["base"] = np.e

sc.pp.highly_variable_genes(adata_T, n_top_genes=3000, batch_key="sample")
adata_T.raw = adata_T
adata_T = adata_T[:, adata_T.var.highly_variable]

sc.pp.scale(adata_T, max_value=10)
sc.pp.pca(adata_T, n_comps=30)
sp.pp.harmony_integrate(adata_T, key="sample", max_iter_harmony=30, verbose=True)
sc.pp.neighbors(adata_T, use_rep="X_pca_harmony")
sc.tl.umap(adata_T)

In [ ]:
sc.tl.leiden(adata_T)

In [ ]:
sc.pl.umap(adata_T, color=['leiden',"CD3E", "MKI67"], size=10, cmap=Reds, legend_loc = 'on data', frameon=False)

In [ ]:
markers_from_article = ['MKI67', "CD8A", "CD4", "IL7R", "GZMK", "KLF2", "CD69", 
                        "XCL1", "SELL", "HSPH1", "LAYN", "GZMA", 
                        "MAL", "TOP2A", "MCM5", "TNFRSF18", "IL32", 
                        "GZMH", "ISG15", "SESN1", "CCR8", "FCGR3A"]

In [ ]:
sc.pl.umap(adata_T, color=["leiden"]+markers_from_article, ncols = 2, size=10, cmap=Reds, legend_loc = 'on data', legend_fontoutline=2)

## Cell type annotation in groups of CD4+ and CD8+ lymphocytes

For a more precise annotation of cell types, I will annotate them separately in the **CD4+** and **CD8+** lymphocyte groups.

Groups of dividing cells and cells with an interferon signal will be taken out separately

In [ ]:
cell_type_cluster_map = {'0': "CD4",
                         '1': "CD4", 
                         '2': "CD4",
                         '3': "CD4",
                         '4': "CD8",
                         '5': "CD4",
                         '6': "CD4",
                         '7': "CD8",
                         '8': "CD8",
                         '9': "CD8",
                         '10': "CD4",
                         '11': "CD4/8-cycling", #маркер MKI67
                         '12': "CD8",
                         '13': "CD4",
                         '14': "CD8",
                         '15': "CD4/8-ISG15-IFN",
                         '16': 'CD8'
                        }

In [ ]:
adata_T.obs = adata_T.obs.assign(CD4_8=adata_T.obs.leiden)
adata_T.obs["CD4_8"] = adata_T.obs["CD4_8"].map(cell_type_cluster_map)

In [ ]:
adata_T.obs.groupby("CD4_8").count()

In [ ]:
adata_CD4 = adata_T[adata_T.obs.CD4_8.isin(['CD4'])].copy()
adata_CD4 = adata_CD4.raw.to_adata()
adata_CD4.uns["log1p"]["base"] = np.e

sc.pp.highly_variable_genes(adata_CD4, n_top_genes=3000, batch_key="sample")
adata_CD4.raw = adata_CD4
adata_CD4 = adata_CD4[:, adata_CD4.var.highly_variable]

sc.pp.scale(adata_CD4, max_value=10)
sc.pp.pca(adata_CD4, n_comps=30)
sc.external.pp.harmony_integrate(adata_CD4, key="sample", max_iter_harmony=30, verbose=True)
sc.pp.neighbors(adata_CD4, use_rep="X_pca_harmony")
sc.tl.umap(adata_CD4)

sc.tl.leiden(adata_CD4)

In [ ]:
sc.pl.umap(adata_CD4, color=['leiden'], size=15, cmap=Reds, legend_loc = 'on data')

In [ ]:
markers_from_article_CD4 = ['MKI67', "CD8A", "CD4", "IL7R", "CD69", 
                            "SELL", "HSPH1", "GZMA", 
                            "MAL", "TNFRSF18", "IL32", "ISG15", "SESN1", "CCR8"]

In [ ]:
sc.pl.umap(adata_CD4, color=['leiden']+markers_from_article_CD4[0:3], size=15, ncols = 2, cmap=Reds, legend_loc = 'on data')

Cells from cluster 10 obviously CD8+

In [ ]:
adata_T.obs.loc[adata_CD4.obs.index[adata_CD4.obs.leiden.isin(['10'])], 'CD4_8'] = 'CD8'

#### CD4 annotation

In [ ]:
adata_CD4 = adata_T[adata_T.obs.CD4_8.isin(['CD4'])]
adata_CD4 = adata_CD4.raw.to_adata()
adata_CD4.uns["log1p"]["base"] = np.e

sc.pp.highly_variable_genes(adata_CD4, n_top_genes=3000, batch_key="sample")
adata_CD4.raw = adata_CD4
adata_CD4 = adata_CD4[:, adata_CD4.var.highly_variable]

sc.pp.scale(adata_CD4, max_value=10)
sc.pp.pca(adata_CD4, n_comps=30)
sc.external.pp.harmony_integrate(adata_CD4, key="sample", max_iter_harmony=30, verbose=True)
sc.pp.neighbors(adata_CD4, use_rep="X_pca_harmony")
sc.tl.umap(adata_CD4)

sc.tl.leiden(adata_CD4)

In [ ]:
sc.pl.umap(adata_CD4, color=['leiden']+markers_from_article_CD4, size=15, ncols = 2, cmap=Reds, legend_loc = 'on data')

In [ ]:
markers = 'FCGR3A FGFBP2 CX3CR1 NKG7 KLF2 TNFSF9 CRTAM GZMK ITM2C CMC1 XCL1 KLRC1 XCL2 GZMH LAYN GZMB VCAM1 KLRD1 SLC4A10 CEBPD KLRB1 IFNGR1 CCL20 TRDC TRDV1 HIST1H4C TOP2A MKI67 TUBB MCM5 TYMS ISG15 MX1 IFI44L IFIT1 IFI6 SELL CCR7 SOCS3 IL7R ANXA1 HSPH1 DNAJB1 NR4A1 HSPA1A HSPA6 CD69 CD40LG FOS TNF GZMA PLIN2 SESN1 CH25H CXCL13 PTPN13 ZBED2 IL21 IL32 CTLA4 FOXP3 IL2RA MAL PMAIP1 ICA1 MAGEH1 PMCH CCR8 TNFRSF4 TNFRSF18'.split()

In [ ]:
sc.pl.heatmap(adata_CD4, markers, groupby='leiden', swap_axes=True, show_gene_labels=True)

In [ ]:
cell_type_cluster_map_CD4 = {'0': "CD4-IL32_Tregs",  
                             '1': "CD4-HSPH1_memory", 
                             '2': "CD4-CD69_activated_memory",
                             '3': "CD4-TNFRSF18_Tfh",
                             '4': "CD4-SELL_naive",
                             '5': "CD4-MAL_Tregs",
                             '6': "CD4-SELL_naive", 
                             '7': "CD4-SESN1_Tfh", 
                             '8': "CD4-GZMA_effectors", 
                             '9': "CD4-IL7R_memory",
                             '10': "CD4-CCR8_Tregs"
                        }

In [ ]:
sc.tl.rank_genes_groups(adata_CD4, groupby="leiden")
sc.pl.rank_genes_groups(adata_CD4, n_genes=10, sharey=False, ncols = 3,  fontsize=20)

In [ ]:
adata_CD4.obs = adata_CD4.obs.assign(cell_type=adata_CD4.obs.leiden)
adata_CD4.obs["cell_type"] = adata_CD4.obs["cell_type"].map(cell_type_cluster_map_CD4)

In [ ]:
adata_CD4.obs

#### CD8 annotation

In [ ]:
adata_CD8 = adata_T[adata_T.obs.CD4_8.isin(['CD8'])]
adata_CD8 = adata_CD8.raw.to_adata()
adata_CD8.uns["log1p"]["base"] = np.e

sc.pp.highly_variable_genes(adata_CD8, n_top_genes=3000, batch_key="sample")
adata_CD8.raw = adata_CD8
adata_CD8 = adata_CD8[:, adata_CD8.var.highly_variable]

sc.pp.scale(adata_CD8, max_value=10)
sc.pp.pca(adata_CD8, n_comps=30)
sc.external.pp.harmony_integrate(adata_CD8, key="sample", max_iter_harmony=30, verbose=True)
sc.pp.neighbors(adata_CD8, use_rep="X_pca_harmony")
sc.tl.umap(adata_CD8)

sc.tl.leiden(adata_CD8)

In [ ]:
sc.pl.umap(adata_CD8, color=['leiden'], size=20, cmap=Reds, legend_loc = 'on data')

In [ ]:
sc.pl.heatmap(adata_CD8, markers, groupby='leiden', swap_axes=True, show_gene_labels=True)

In [ ]:
markers_from_article_CD8 = ['MKI67', "CD8A", "CD4", "CD3E", "GZMK", "KLF2", "CD69", 
                        "XCL1", "HSPH1", "LAYN", 
                        "GZMH", "ISG15", "FCGR3A", "TRDC"]

In [ ]:
sc.pl.umap(adata_CD8, color=['leiden', 'SELL']+markers_from_article_CD8, size=15, ncols = 2, cmap=Reds, legend_loc = 'on data')

In [ ]:
cell_type_cluster_map_CD8 = { '0': "CD8-GZMH_transitional",
                             '1': "CD8-GZMK_circulating_precursors", 
                             '2': "CD8-LAYN_terminally_differentiated",
                             '3': "CD8-KLF2_circulating_precursors",
                             '4': "CD8-XCL1_resident_precursors",
                             '5': "CD8-FCGR3A_effectors",
                             '6': "CD8-LAYN_terminally_differentiated", 
                             '7': "CD8-SELL_naive",
                             '8': "CD8-SELL_naive",
                             '9': "CD8-TRDC-gamma_delta",
                             '10': "CD8-SLC4A10_MAIT",
                        }

In [ ]:
sc.tl.rank_genes_groups(adata_CD8, groupby="leiden")

In [ ]:
sc.pl.rank_genes_groups(adata_CD8, n_genes=10, sharey=False, ncols = 3,  fontsize=20)

In [ ]:
adata_CD8.obs = adata_CD8.obs.assign(cell_type=adata_CD8.obs.leiden)
adata_CD8.obs["cell_type"] = adata_CD8.obs["cell_type"].map(cell_type_cluster_map_CD8)

#### Write the names of cell types in the original dataset in the cell_type column

In [ ]:
adata_T.obs = adata_T.obs.merge(pd.concat([adata_CD8.obs.cell_type, adata_CD4.obs.cell_type]), how = "left", left_index=True, right_index=True)
adata_T.obs['cell_type'] = adata_T.obs['cell_type'].fillna(adata_T.obs.pop('CD4_8'))
adata_T.obs

In [ ]:
sc.pl.umap(adata_T, color=['leiden', "cell_type"], size=10, cmap=Reds, ncols = 2, legend_loc = 'right margin')

## TCR analysis

Create summary about the Immune cell-receptor compositions and visualise

In [ ]:
ir.tl.chain_qc(adata_T)

In [ ]:
ax = ir.pl.group_abundance(adata_T, groupby="receptor_type", target_col="source")

In [ ]:
ax = ir.pl.group_abundance(adata_T, groupby="receptor_subtype", target_col="source")

In [ ]:
ax = ir.pl.group_abundance(adata_T, groupby="chain_pairing", target_col="source")

In [ ]:
ax = ir.pl.group_abundance(adata_T, groupby="chain_pairing", target_col="patient")

### Filter cells based on TCR

In [ ]:
adata_TCR_filtered = adata_T[adata_T.obs["chain_pairing"] != "multichain", :].copy()
adata_TCR_filtered = adata_TCR_filtered[~adata_TCR_filtered.obs["chain_pairing"].isin(["orphan VDJ", "orphan VJ", 'no IR', 'ambiguous']), :].copy()

In [ ]:
ax = ir.pl.group_abundance(adata_TCR_filtered, groupby="chain_pairing", target_col="source")

In [ ]:
ax = ir.pl.group_abundance(adata_TCR_filtered, groupby="chain_pairing", target_col="patient")

### Find clonotypes

In [ ]:
ir.pp.ir_dist(adata_TCR_filtered)
ir.tl.define_clonotypes(adata_TCR_filtered, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.tl.clonotype_network(adata_TCR_filtered, min_cells=15)

In [ ]:
ir.pl.clonotype_network(adata_TCR_filtered, color="source" , base_size=20, label_fontsize=9, panel_size=(7, 7))

In [ ]:
adata_TCR_filtered.obs

In [ ]:
plt.figure(figsize=(15,5))
ax = sns.histplot(adata_TCR_filtered.obs.clone_id_size, binwidth=1)
ax.set_xlim(2, 105)
ax.set_ylim(0, 2600)

Write infornation abour TCRs in dataset with all T cells

In [ ]:
adata_T.obs = adata_T.obs.merge(adata_TCR_filtered.obs[['clone_id','clone_id_size']], how = "left", left_index=True, right_index=True)

In [ ]:
adata_T.obs

In [ ]:
sc.pl.umap(adata_T, color=["clone_id_size", 'MKI67', "cell_type"], size = 15, cmap = Reds, ncols = 2)

In [ ]:
sc.set_figure_params(figsize = (20, 15))

### Creating a table with the probabilities of hitting a particular cell type
(in a clonotype from 10 cells and at least one of the cells is in a cluster of dividing cells)

In [ ]:
clonotypes_10 = adata_T.obs[['cell_type', 'clone_id','clone_id_size']][adata_T.obs[['cell_type', 'clone_id','clone_id_size']]["clone_id_size"] > 10].sort_values('clone_id')
clonotypes_10

In [ ]:
clonotypes_10_cycling = clonotypes_10[clonotypes_10.clone_id.isin(clonotypes_10[clonotypes_10['cell_type']=='CD4/8-cycling'].clone_id)]
clonotypes_10_cycling

In [ ]:
cell_types = list(set(clonotypes_10_cycling.cell_type))
cell_types.remove('CD4/8-cycling')
cell_types = list(set(clonotypes_10_cycling.columns)) + ['CD4/8-cycling'] + cell_types
cell_types.remove('cell_type')

In [ ]:
clonotype_cell_type = pd.DataFrame(columns = cell_types)
for clone in list(set(clonotypes_10_cycling.clone_id)):
    clonotype_cell_type.loc[len(clonotype_cell_type.index)] = np.zeros(len(cell_types))
    temp = clonotypes_10_cycling[clonotypes_10_cycling.clone_id == clone]
    
    clonotype_cell_type['clone_id_size'][len(clonotype_cell_type.index)-1] = temp.clone_id_size[0]
    clonotype_cell_type['clone_id'][len(clonotype_cell_type.index)-1] = temp.clone_id[0]
    
    for cell in temp.index:
        clonotype_cell_type[temp.loc[cell].cell_type][len(clonotype_cell_type.index)-1]+= 1
    
clonotype_cell_type    

In [ ]:
clonotype_cell_type_prop = clonotype_cell_type.copy()
for ct in clonotype_cell_type_prop.columns[3:]:
    clonotype_cell_type_prop[ct] = clonotype_cell_type_prop[ct] / (clonotype_cell_type_prop['clone_id_size'] - clonotype_cell_type_prop['CD4/8-cycling'])
    
clonotype_cell_type_prop

In [ ]:
clonotype_cell_type_prop.to_csv("data/GSE162500_cell_type_ver.csv")

In [ ]:
sc.pl.umap(adata_T, color="cell_type", show=False, size=30)
sc.pl.umap(
    adata_T,
    color="clone_id",
    groups = list(clonotype_cell_type.clone_id),
    palette=cycler(color=mpl_cm.Dark2_r.colors), 
    size=80
)

In [ ]:
ir.pl.group_abundance(adata_T[adata_T.obs.clone_id.isin(clonotype_cell_type.clone_id)], groupby="clone_id", target_col="cell_type")

## Preparing a dataset for validation

It is necessary to make a dataset with counts and information about cell types, clonotypes, and so on.

**adata_T.obs** has the necessary information, and **adata** (downloaded from data/GSE162500_adata_all_samples_filtered.h5ad) has count information, but it contains data on poor-quality UMI and non-T cells

In [ ]:
#индексы нужных клеток
adata_T.obs.index

In [ ]:
adata = sc.read("data/GSE162500_adata_all_samples_filtered.h5ad")

Keep only the good T cells

In [ ]:
adata_val = adata[adata.obs.index.isin(adata_T.obs.index)]

In [ ]:
adata_val.obs

In [ ]:
adata_val.obs = adata_T.obs

In [ ]:
adata_val.obs

In [ ]:
adata_val.write("data/GSE162500_for_validation.h5ad")